In [6]:
import pandas as pd
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
import pickle
import time, datetime, os
%reload_ext tensorboard

In [7]:
NAME = "264x32x16x1_MSE_sv{}".format(int(time.time()))
log_dir = "logs/fit/" + NAME
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
stats = pd.read_csv(r'Data\final_dataset.csv')
#stats = stats.drop('Score',1)
#stats['Results'] = pd.to_numeric(stats['Results'], errors='coerce')
stats['Results'] = stats['Results'].astype('float32')


In [8]:
X = stats.iloc[:,3:-4]
y = stats.iloc[:,-1:]
random_state = 12
#split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = True, random_state=random_state)
X_train = tf.keras.utils.normalize(X_train, axis=1)
X_test = tf.keras.utils.normalize(X_test, axis=1)
odds1 = stats.iloc[:, -3:-2].values
odds1_prob = 1/odds1
odds1_prob
odds2 = stats.iloc[:, -2:-1].values


In [9]:
from tensorflow.python.ops import math_ops

def coef_bce(y_pred,y_true, odds1, odds2):
    return -1 * K.mean((odds1 * y_true -1) * K.log(odds1 * y_pred -1) + (odds2 * (1- y_true) -1) * K.log(odds2 * (1-y_pred)-1))

def bce_custom(odds1, odds2):
    def bce(y_pred, y_true):
        return coef_bce(y_pred, y_true, odds1, odds2)
    return bce

loss_bce = bce_custom(odds1, odds2)

def coef_mse(y_pred, y_true, odds_game):
    #mse = K.square(y_pred - y_true)  #squared difference
    #odds = 0.05 * K.square(y_pred - odds_game)   
    #loss = K.mean(mse - odds , axis=-1) #mean over last dimension
    return K.mean(K.square(y_pred - y_true) + 0.10 * K.square(y_pred- odds_game), axis=-1)
def mse_custom(odds_game):
    def mse(y_true, y_pred):
        return coef_mse(y_pred, y_true, odds_game )
    return mse


loss_mse = mse_custom(odds1_prob)

def mse_simple(y_true, y_pred):
    mse = math_ops.squared_difference(y_pred, y_true) 
    loss = K.mean(mse, axis=-1)
    return loss
    

In [10]:
#opt = tf.keras.optimizers.Adam(clipnorm=0.3)

model = Sequential()
#model.add(tf.keras.layers.Flatten())
model.add(Dense(264, input_dim=264, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(32, kernel_initializer='normal', activation='relu', bias_regularizer=l2(0.02)))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(optimizer='Adam', loss=loss_mse, metrics=['accuracy'])
model.fit(X_train, y_train, batch_size= 1, epochs=5, validation_data=(X_test, y_test),callbacks=[tensorboard_callback])

Epoch 1/5
11440/11440 [==============================] - 7s 631us/step - loss: 0.2295 - accuracy: 0.6455 - val_loss: 0.2194 - val_accuracy: 0.6598
Epoch 2/5
11440/11440 [==============================] - 7s 575us/step - loss: 0.2253 - accuracy: 0.6559 - val_loss: 0.2191 - val_accuracy: 0.6633
Epoch 3/5
11440/11440 [==============================] - 7s 573us/step - loss: 0.2233 - accuracy: 0.6605 - val_loss: 0.2176 - val_accuracy: 0.6629
Epoch 4/5
11440/11440 [==============================] - 7s 579us/step - loss: 0.2210 - accuracy: 0.6649 - val_loss: 0.2192 - val_accuracy: 0.6710
Epoch 5/5
11440/11440 [==============================] - 7s 573us/step - loss: 0.2198 - accuracy: 0.6661 - val_loss: 0.2193 - val_accuracy: 0.6766


In [309]:
#val_loss, val_acc = model.evaluate(X_test, y_test)

In [310]:
prediction = model.predict(X_test[:])
label = y_test[:]
prediction_l = prediction.tolist()
pred =pd.DataFrame(prediction_l, columns=['prediction'])


In [311]:
stats.reset_index
index = stats[['GAME_DATE', 'HOME', 'AWAY','Odds1', 'Odds2', 'Score']]
data = label.join(index)
data.reset_index(inplace=True)
pred_data = data.join(pred)
pred_data.to_csv('predicted_data_custom.csv')